In [1]:
from src.code_gen.code_generator import CCodeGenerator
from src.lexer.hulk_lexer import HulkLexer
from src.parser.hulk_parser import HulkParser
from src.hulk_grammar.hulk_grammar import G
from src.evaluation import evaluate_reverse_parse
from src.semantics.semantic_analysis_pipeline import semantic_analysis_pipeline

lexer = HulkLexer(True)
parser = HulkParser()

In [10]:
tokens, errors = lexer('''      
    type D {
        getc() => while(false) 5;
        equals(other: Object) => true;
    }
                       
    let a = new D() in
        if (3 as Object == "abc" as Object)
            print(a.getc())
        else
            print(false);
    ''')

derivation, operations, parse_errors = parser(tokens)
print(parse_errors)

if len(parse_errors) == 0:
    ast = evaluate_reverse_parse(derivation, operations, tokens)
    ast, semantic_errors, context, scope = semantic_analysis_pipeline(ast, False)

    print(semantic_errors)

    if len(semantic_errors) == 0:
        generator = CCodeGenerator()
        code = generator(ast, context)

        with open("test.c", "w") as out:
            out.write(code)

[]
[]


In [11]:
import subprocess

subprocess.run(["gcc", "test.c", "-o", "test.exe"], shell=True)
subprocess.run(['start', 'cmd', '/k', 'test.exe'], shell=True)

CompletedProcess(args=['start', 'cmd', '/k', 'test.exe'], returncode=0)